In [6]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [7]:
mlflow.set_experiment("EXP_5-Xgboost_hpt_1000")

<Experiment: artifact_location='s3://apoorv-mlfow-bucket/285964941007082429', creation_time=1758734578723, experiment_id='285964941007082429', last_update_time=1758734578723, lifecycle_stage='active', name='EXP_5-Xgboost_hpt_1000', tags={}>

In [8]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [9]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [14]:
# Step 1: Remap class labels [-1,0,1] -> [2,0,1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows with NaN target
df = df.dropna(subset=['category'])

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# Step 4: TF-IDF Vectorization
ngram_range = (1, 3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Step 5: Handle class imbalance with ADASYN
adasyn = ADASYN(random_state=42)
X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)

# ----------------------------
# MLflow logging function
# ----------------------------
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, best_params):
    with mlflow.start_run():
        # Log experiment info
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        
        # Log best hyperparameters
        mlflow.log_params(best_params)
        
        # Log algorithm name
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# ----------------------------
# Optuna objective function
# ----------------------------
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))

# ----------------------------
# Run Optuna experiment & log best model + params
# ----------------------------
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get best parameters
    best_params = study.best_params

    # Create best model with best parameters
    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        random_state=42
    )

    # Log model, metrics, and best parameters in same MLflow run
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test, best_params)

# ----------------------------
# Run the experiment
# ----------------------------
run_optuna_experiment()

[I 2025-09-25 01:39:09,041] A new study created in memory with name: no-name-23dce193-e690-4570-86da-a00bc06ebfa4
[I 2025-09-25 01:39:11,847] Trial 0 finished with value: 0.8168221010029914 and parameters: {'n_estimators': 256, 'learning_rate': 0.02463876133601436, 'max_depth': 4}. Best is trial 0 with value: 0.8168221010029914.
[I 2025-09-25 01:39:15,612] Trial 1 finished with value: 0.7381664613760338 and parameters: {'n_estimators': 197, 'learning_rate': 0.002568037893815669, 'max_depth': 7}. Best is trial 0 with value: 0.8168221010029914.
[I 2025-09-25 01:39:19,759] Trial 2 finished with value: 0.7779341896885448 and parameters: {'n_estimators': 221, 'learning_rate': 0.007657545106474515, 'max_depth': 7}. Best is trial 0 with value: 0.8168221010029914.
[I 2025-09-25 01:39:24,643] Trial 3 finished with value: 0.7386943515748724 and parameters: {'n_estimators': 230, 'learning_rate': 0.0020966806989837685, 'max_depth': 8}. Best is trial 0 with value: 0.8168221010029914.
[I 2025-09-25 

🏃 View run XGBoost_ADASYN_TFIDF_Trigrams at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/285964941007082429/runs/cf0ed3b66d044ceb9fb750194428f2ac
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/285964941007082429


EndpointConnectionError: Could not connect to the endpoint URL: "https://apoorv-mlfow-bucket.s3.eu-north-1b.amazonaws.com/285964941007082429/models/m-4f1156f7c6a343cf8cd1bb594c0f810e/artifacts/conda.yaml"

In [11]:
import boto3

s3 = boto3.client("s3", region_name="eu-north-1")
response = s3.list_buckets()
print([bucket['Name'] for bucket in response['Buckets']])

['apoorv-mlfow-bucket']


In [10]:
model_uri="http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/650043532633511350/models/m-705c883531e64a6fbae8f09999170671"
loaded_model = mlflow.sklearn.load_model(model_uri)

ValueError: not enough values to unpack (expected 2, got 1)

In [12]:
import mlflow.sklearn

# Your actual run ID
run_id = "1d05c19a188945adafb55c17d8f950c0"

# Artifact path — must match what you used in log_mlflow
# In your code it was: mlflow.sklearn.log_model(model, f"{model_name}_model")
# So the folder should be "XGBoost_model"
model_uri = f"runs:/{run_id}/XGBoost_model"

In [13]:
loaded_model = mlflow.sklearn.load_model(model_uri)

# Extract the parameters used by this trained model
print("Best parameters used in this model:")
for k, v in loaded_model.get_params().items():
    print(f"{k}: {v}")

Best parameters used in this model:
objective: multi:softprob
base_score: None
booster: None
callbacks: None
colsample_bylevel: None
colsample_bynode: None
colsample_bytree: None
device: None
early_stopping_rounds: None
enable_categorical: False
eval_metric: None
feature_types: None
feature_weights: None
gamma: None
grow_policy: None
importance_type: None
interaction_constraints: None
learning_rate: 0.09087294404864565
max_bin: None
max_cat_threshold: None
max_cat_to_onehot: None
max_delta_step: None
max_depth: 10
max_leaves: None
min_child_weight: None
missing: nan
monotone_constraints: None
multi_strategy: None
n_estimators: 264
n_jobs: None
num_parallel_tree: None
random_state: 42
reg_alpha: None
reg_lambda: None
sampling_method: None
scale_pos_weight: None
subsample: None
tree_method: None
validate_parameters: None
verbosity: None


In [15]:
best_params = {
    "learning_rate": loaded_model.get_params()["learning_rate"],
    "max_depth": loaded_model.get_params()["max_depth"],
    "n_estimators": loaded_model.get_params()["n_estimators"],
    "random_state": loaded_model.get_params()["random_state"],
}

print("Best tuned parameters:", best_params)

Best tuned parameters: {'learning_rate': 0.09087294404864565, 'max_depth': 10, 'n_estimators': 264, 'random_state': 42}
